In [ ]:
import numpy as np
import pandas as pd
import pydicom as dicom
import cv2
import os 
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras_preprocessing.image.dataframe_iterator import DataFrameIterator
import matplotlib.pylab as plt
from tensorflow.keras.utils import Sequence
from keras.utils import data_utils
from keras.applications.vgg19 import VGG19
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
import tensorflow as tf
from keras.layers.convolutional import Conv2D
from keras.layers.core import Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
import pandas as pd
from glob import glob

In [ ]:
train_df = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv')
test_df = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv')
train_array = np.array(train_df['BraTS21ID'])
train_label =  np.array(train_df['MGMT_value'])
test_array = np.array(test_df['BraTS21ID'])
test_label =  np.array(test_df['MGMT_value'])
DIM = 512
NB_CHANNELS = 1
i = 0
BATCH_SIZE = 32
a = 0
train_ds = pd.DataFrame(columns = ['path','label'] )
test_ds = pd.DataFrame(columns = ['path','label'] )

In [ ]:
len(train_df)

In [ ]:
test_df

In [ ]:
print(str(10).zfill(5))

In [ ]:
dataframe_values = []
for a,b in zip(train_df['BraTS21ID'],train_df['MGMT_value']):
    folders_list = os.listdir(f'../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{str(a).zfill(5)}')
    for folder in folders_list :  
        img_list = os.listdir(f'../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{str(a).zfill(5)}/{folder}')
        for img in img_list : 
            dataframe_values.append({'path' : f'../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{str(a).zfill(5)}/{folder}/{img}','label' : b})
train_ds  = pd.DataFrame.from_dict(dataframe_values)



In [ ]:
val_ds = train_ds[300000:].reset_index()
train_ds = train_ds[:300000]

In [ ]:
train_ds

# unlabeled Data

In [ ]:
dataframe_values = []
for a,b in zip(test_df['BraTS21ID'],test_df['MGMT_value']):
    folders_list = os.listdir(f'../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/{str(a).zfill(5)}')
    for folder in folders_list :    
        img_list = os.listdir(f'../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/{str(a).zfill(5)}/{folder}')
        for img in img_list : 
            dataframe_values.append({'path' : f'../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/{str(a).zfill(5)}/{folder}/{img}','label' : b})
test_ds  = pd.DataFrame.from_dict(dataframe_values)


In [ ]:
test_ds

In [ ]:
"""train_augmentation_parameters = dict(
    rescale=1.0/255.0,
    rotation_range=10,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range = [0.8, 1.2])

valid_augmentation_parameters = dict(
    rescale=1.0/255.0)

# Using the training phase generators 
train_augmenter = ImageDataGenerator(**train_augmentation_parameters)
valid_augmenter = ImageDataGenerator(**valid_augmentation_parameters)"""

# in this part of code , i use keras dataGenerator  ,the problem here is that it generates the whole folder, so the first layer input for example is [900,512,512,1] which is going to give an OOM Error , so the solution here is to make a 16 (just an example ) FramesDatagenerator

In [ ]:
"""class Frames_Generator(data_utils.Sequence):
    'Generates data for Keras'
    def __init__(self,Sample_array, batch_size=1, dim=(512,512), shuffle=True,  train = True ):
        'Initialization'
        self.batch_size = batch_size
        self.Sample_array = Sample_array 
        self.shuffle = shuffle
        self.dim = dim
        self.train = train
        self.on_epoch_end()
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.Sample_array) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [str(self.Sample_array[k]).zfill(5) for k in indexes]
        #list_IDs_temp = [20]
        # Generate data
        X, y = self.__data_generation(list_IDs_temp, index)
        X =  np.array([X])[0]
        y =  np.array([y])
        y = y.reshape(y.shape[1])
        randomize = np.arange(y.shape[0])
        np.random.shuffle(randomize)
        X = X[randomize]
        y = y[randomize]
        #train_generator = train_augmenter.flow(X ,y,batch_size = 32 )
        return X, y
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.Sample_array))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp,index):
        label_train  = []
        Frame_train = [] 
        global a 
        global b
        print(list_IDs_temp[0])
        if self.train :
            folders_list = os.listdir(f'../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{list_IDs_temp[0]}')
            for folder in folders_list : 
                img_list = os.listdir(f'../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{list_IDs_temp[0]}/{folder}')
                for img in img_list :
                    ds = dicom.dcmread(f'../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{list_IDs_temp[0]}/{folder}/{img}').pixel_array
                    img = cv2.resize(ds,self.dim)
                    img = img.reshape(self.dim[0],self.dim[1],1)
                    label_train.append(train_label[index])
                    Frame_train.append(img)
        else : 
            folders_list = os.listdir(f'../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/{list_IDs_temp[0]}')
            for folder in folders_list : 
                img_list = os.listdir(f'../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/{list_IDs_temp[0]}/{folder}')
                for img in img_list :
                    ds = dicom.dcmread(f'../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/{list_IDs_temp[0]}/{folder}/{img}').pixel_array
                    img = cv2.resize(ds,self.dim)
                    img = img.reshape(dim[0],dim[1],1)
                    label_train.append(test_label[index])
                    Frame_train.append(img)    
        return Frame_train, label_train"""

In [ ]:
"""# Parameters
params = {'dim': (512,512),
          'batch_size': 1,
          'shuffle': True, 
           'train' : True }
valid_params = {'dim': (512,512),
          'batch_size': 1,
          'shuffle': True,
          'train':False}
training_generator = Frames_Generator(train_array, **params)
validation_generator = Frames_Generator(test_array, **valid_params)"""


# part2 

In [ ]:
class Frames_Generator(data_utils.Sequence):
    'Generates data for Keras'
    def __init__(self,Sample_df = np.arange(len(train_ds)//BATCH_SIZE-32) , batch_size=1, dim=(512,512), shuffle=True ,train_ds = train_ds,nb_steps = 0 ):
        'Initialization'
        self.batch_size = batch_size
        self.Sample_df = Sample_df
        self.shuffle = shuffle
        self.dim = dim
        self.train_ds = train_ds
        self.nb_steps = nb_steps
        self.on_epoch_end()
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.Sample_df) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [str(self.Sample_df[k]).zfill(5) for k in indexes]
        #list_IDs_temp = [20]
        # Generate data
        X, y = self.__data_generation(list_IDs_temp)
        X =  np.array([X])[0]
        y =  np.array([y])
        y = y.reshape(y.shape[1])
        randomize = np.arange(y.shape[0])
        np.random.shuffle(randomize)
        X = X[randomize]
        y = y[randomize]
        #train_generator = train_augmenter.flow(X ,y,batch_size = 32 )
        return X/255, y
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.Sample_df))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        label_train  = []
        Frame_train = [] 
        i = 0
        while(i<32) : 
            ds = dicom.dcmread(self.train_ds['path'][self.nb_steps]).pixel_array
            img = cv2.resize(ds,self.dim)
            img = img.reshape(self.dim[0],self.dim[1],1)
            
            label = self.train_ds['label'][self.nb_steps]
            label_train.append(self.train_ds['label'][self.nb_steps])
            Frame_train.append(img)
            self.nb_steps += 1 
            i +=1
            if  self.nb_steps == len(self.train_ds) :
                self.nb_steps = 0
        return Frame_train, label_train

In [ ]:
len(train_ds)//BATCH_SIZE-32

In [ ]:
# Parameters
train_params = {'Sample_df':np.arange(len(train_ds)//BATCH_SIZE-32),
          'dim': (512,512),
          'batch_size': 1,
          'shuffle': True, 
           'train_ds' : train_ds,
         'nb_steps': 0}
test_params = {'Sample_df':np.arange(len(test_ds)//BATCH_SIZE-32),
          'dim': (512,512),
          'batch_size': 1,
          'shuffle': True, 
           'train_ds' : test_ds,
         'nb_steps': 0}
val_params = {'Sample_df':np.arange(len(val_ds)//BATCH_SIZE-32),
          'dim': (512,512),
          'batch_size': 1,
          'shuffle': True, 
           'train_ds' : val_ds,
         'nb_steps': 0}
training_generator = Frames_Generator(**train_params)
test_generator = Frames_Generator(**val_params)
validation_generator = Frames_Generator(**test_params)


In [ ]:
"""x , y = validation_generator.__getitem__(0)"""

In [ ]:
arr = np.arange(len(train_ds)//BATCH_SIZE)
len(arr)

# Baseline model

In [ ]:

model = Sequential()
model.add(Conv2D(32, (3, 3), padding="same",input_shape = (DIM , DIM , NB_CHANNELS)))
model.add(Conv2D(32, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(BatchNormalization(axis=1))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(BatchNormalization(axis=1))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(BatchNormalization(axis=1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(BatchNormalization(axis=1))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(BatchNormalization(axis=1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1024))
model.add(Dropout(0.2))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dense(512))
model.add(Dropout(0.2))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dense(64))
model.add(Dropout(0.2))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dense(16))
model.add(Dropout(0.2))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dense(1))
model.add(Activation("sigmoid"))
model.build((0,512,512,1))
model.summary()

# VGG19 Model 

In [ ]:

# Create the base model of VGG19
vgg19 = VGG19( include_top=False, input_shape = (512, 512, 3) , weights = 'imagenet')

In [ ]:
vgg19.summary()


In [ ]:
vgg = Sequential()
vgg.add(Conv2D(3, (3, 3), padding="same",input_shape = (DIM , DIM , NB_CHANNELS)))
for layer in vgg19.layers[1:-1]: # this is where I changed your code
    vgg.add(layer)    
# Freeze the layers 
#for layer in model.layers:
#    layer.trainable = False

In [ ]:

vgg.add(Flatten())
vgg.add(Dense(64 , activation = 'relu'))
vgg.add(Dense(32 , activation = 'relu'))
vgg.add(Dense(1, activation='sigmoid'))

In [ ]:
vgg.summary()


In [ ]:
from keras.callbacks import ReduceLROnPlateau

learning_rate_reduction = ReduceLROnPlateau(monitor='accuracy',
                                            patience = 1,
                                            verbose=1,
                                            factor=0.1,
                                            min_lr=0.000001)

opt = tf.keras.optimizers.Adam(learning_rate=0.01)

model.compile(optimizer = opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(training_generator,validation_data = test_generator , epochs = 4,callbacks = [learning_rate_reduction])

In [ ]:
model.save("./model.h5")


In [ ]:
#model = tf.keras.models.load_model('../input/tumor-model/model.h5')

In [ ]:
result = model.predict(validation_generator, verbose = True, workers = 2)


In [ ]:
result.shape

In [ ]:
data_dir = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/test'
patients_test = sorted(os.listdir(data_dir))

In [ ]:
patients_test

In [ ]:
final_result = []
i = 0
for patient in patients_test:
    arr = os.listdir('../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/' + patient)
    length = 0
    for j in arr : 
        lenn = len(glob('../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/' + patient +'/'+j+'/*.dcm'))
        length = lenn + length 
    final_result.append(result[i:i+length].sum()/length)
    i += length
    


In [ ]:
final_result

In [ ]:
test = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv')


In [ ]:
submission = pd.DataFrame({"BraTS21ID": test['BraTS21ID'].apply(lambda x: str(x).zfill(5)), "MGMT_value": final_result})
submission


In [ ]:
submission.to_csv('submission.csv', index = 0)